In [21]:
import matplotlib
matplotlib.use("TkAgg") #backend
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg,NavigationToolbar2TkAgg
from matplotlib.figure import Figure

In [2]:
import tkinter as tk
from tkinter import ttk #for good looking buttons

In [3]:
LARGE_FONT= ("Verdana",12)

In [4]:
class seaofBTCapp(tk.Tk): # inherit the attributes of another class 
    
    def __init__(self, *args,**kwargs): 
        # initialize with your class
        # *args- short form of arguments, you can pass any number of variables
        # **kwargs- keyword arguments, passing through dictionaries 
        
        tk.Tk.__init__(self, *args, **kwargs)
        
#         tk.Tk.iconbitmap(self,default="ball.xbm")
        tk.Tk.wm_title(self,"Sea of BTC client")
        container = tk.Frame(self)
        # contain everything of our tkinter app ,or edge of window
        container.pack(side="top", fill="both", expand= True)
        # pack and grid are two option to arrange
        
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames ={}
        
        for F in (StartPage,PageOne,PageTwo,PageThree):

            frame = F(container,self)

            self.frames[F] = frame

            frame.grid(row=0,column=0,sticky="nsew")
        
        self.show_frame(StartPage)
        
    def show_frame(self,cont):
        
        frame = self.frames[cont]
        frame.tkraise()

In [5]:
class StartPage(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Start Page", font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Visit Page 1",
                           command=lambda: controller.show_frame(PageOne))
        button1.pack()
        
        button2 = ttk.Button(self, text="Visit Page 2",
                           command=lambda: controller.show_frame(PageTwo))
        button2.pack()
        
        button3 = ttk.Button(self, text="Graph Page",
                           command=lambda: controller.show_frame(PageThree))
        button3.pack()

In [6]:
class PageOne(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Page One", font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Back to home",
                           command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        button2 = ttk.Button(self, text="Page two",
                   command=lambda: controller.show_frame(PageTwo))
        button2.pack()

In [7]:
class PageTwo(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Page Two", font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Back to home",
                           command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        button2 = ttk.Button(self, text="Page One",
                   command=lambda: controller.show_frame(PageOne))
        button2.pack()

In [24]:
class PageThree(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Graph Page", font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Back to home",
                           command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        f = Figure(figsize=(5,5),dpi=100)
        a = f.add_subplot(111)
        a.plot([1,2,3,4,5,6,7,8],[3,4,2,4,-3,9,1,0])
        
        canvas = FigureCanvasTkAgg(f,self)
        canvas.show()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand= True)
        
        tollbar = NavigationToolbar2TkAgg(canvas,self)
        tollbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand= True)

In [26]:
app = seaofBTCapp()
app.mainloop()